In [ ]:
# check if notebook is in colab
try:
    # install ezkl
    import google.colab
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "ezkl"])
    subprocess.check_call([sys.executable, "-m", "pip", "install", "onnx"])

# rely on local installation of ezkl if the notebook is not in colab
except:
    pass

In [ ]:
from torch import nn
import ezkl
import os
import json
import torch
import math


class WorldEngine(nn.Module):
  def __init__(self):
    super(WorldEngine, self).__init__()

    ## phi will equal 15 degrees of rotation
    self.phi = 15
    self.full_rotation = 360

  def forward(self, theta):
    theta = torch.add(theta, self.phi)
    theta = torch.remainder(theta, 360)
    return theta
  
engine_model = WorldEngine()

This will showcase the principle directions of rotation by plotting the rotation of a single unit vector.

In [ ]:

from matplotlib import pyplot

def degrees_to_radians(degrees):
  return torch.tensor(degrees * math.pi / 180)

def render_arrows(degrees):
  phi_radians = degrees_to_radians(engine_model.phi)

  pyplot.figure(figsize=(3, 3))
  pyplot.arrow(0, 0, 1, 0, width=0.02, alpha=0.5)
  pyplot.arrow(0, 0, 0, 1, width=0.02, alpha=0.5)
  pyplot.arrow(0, 0, torch.cos(phi_radians), torch.sin(phi_radians), width=0.02)
  pyplot.arrow(0, 0, -torch.sin(phi_radians), torch.cos(phi_radians), width=0.02)

render_arrows(engine_model.phi)


In [ ]:
# initial principle vectors positions (degreee 0) for the rotation are as in the plot above
input = torch.zeros(1)

engine_model.eval()

model_path = os.path.join("network.onnx")


torch.onnx.export(
    engine_model,
    input,
    model_path,
    export_params=True,
    opset_version=14,
    do_constant_folding=True,
    input_names=["input"],
    output_names=["output"],
)

In [ ]:
data_path = os.path.join("input.json")
data_array = ((input).detach().numpy()).reshape([-1]).tolist()
data = dict(input_data = [data_array])
json.dump(data, open(data_path, "w"))

In [ ]:
settings_path = os.path.join("settings.json")
srs_path = os.path.join("kzg.srs")
witness_path = os.path.join("witness.json")
compiled_model_path = os.path.join("network.compiled")
pk_path = os.path.join("test.pk")
vk_path = os.path.join("test.vk")

In [ ]:
run_args = ezkl.PyRunArgs()
run_args.input_visibility = "public"
run_args.param_visibility = "fixed"
run_args.output_visibility = "public"
# we set both input and param scale to 0 since we input and params will always be represented as integers. (No precision loss)
run_args.scale_rebase_multiplier = 10
run_args.input_scale = 0
run_args.param_scale = 0

assert ezkl.gen_settings(model_path, settings_path, py_run_args=run_args)

In [ ]:
assert ezkl.compile_circuit(model_path, compiled_model_path, settings_path)

In [ ]:
assert ezkl.get_srs(settings_path)

In [ ]:
witness = ezkl.gen_witness(data_path, compiled_model_path, witness_path)
assert os.path.isfile(witness_path)

In [ ]:
res = ezkl.setup(
  compiled_model_path,
  vk_path,
  pk_path,
  witness_path=witness_path,
)
assert res == True
assert os.path.isfile(vk_path)
assert os.path.isfile(pk_path)
assert os.path.isfile(settings_path)

In [ ]:
# GENERATE A PROOF

proof_path = os.path.join('test.pf')

res = ezkl.prove(
        witness_path,
        compiled_model_path,
        pk_path,
        proof_path,
        "single",
    )

print(res)
assert os.path.isfile(proof_path)

In [ ]:
# VERIFY IT
res = ezkl.verify(
        proof_path,
        settings_path,
        vk_path,
    )

assert res == True
print("verified")

We can now create an EVM / `.sol` verifier that can be deployed on chain to verify submitted proofs using a view function.

In [ ]:
abi_path = 'test.abi'
sol_code_path = 'test_1.sol'

res = ezkl.create_evm_verifier(
        vk_path,
        settings_path,
        sol_code_path,
        abi_path,
    )
assert res == True

## Verify on the evm

In [ ]:
# Make sure anvil is running locally first
# run with $ anvil -p 3030
# we use the default anvil node here
import json

address_path = os.path.join("address.json")

res = ezkl.deploy_evm(
    address_path,
    sol_code_path,
    'http://127.0.0.1:3030'
)

assert res == True

with open(address_path, 'r') as file:
    addr = file.read().rstrip()

In [ ]:
# make sure anvil is running locally
# $ anvil -p 3030

res = ezkl.verify_evm(
    proof_path,
    addr,
    "http://127.0.0.1:3030"
)
assert res == True

As a sanity check lets plot the rotations of the unit vectors. We can see that the unit vectors rotate as expected by the output of the circuit. 

In [ ]:
witness['outputs'][0][0]

In [ ]:
settings = json.load(open(settings_path, 'r'))
out_scale = settings["model_output_scales"][0]


degree_change = ezkl.vecu64_to_float(witness['outputs'][0][0], out_scale)
print(degree_change)
render_arrows(degree_change)